# 處理環境


當您以 Azure Machine Learning 作業身分執行腳本時，您必須定義作業執行的執行內容。 其中一個金鑰組態是執行腳本的計算目標。 這可能是本機工作站 (在此情況下，計算實例) ，或遠端計算目標，例如隨選布建的 Azure Machine Learning 受控計算叢集。

在此筆記本中，您將建立計算叢集，並探索作業的計算目標。

## 開始之前

您將需要最新版的  **azureml-ai-ml** 套件，才能在此筆記本中執行程式碼。 執行下列資料格以確認已安裝它。

> **注意**：
> 如果未安裝 **azure-ai-ml** 套件，請執行 `pip install azure-ai-ml` 以安裝它。

In [ ]:
## 連線到您的工作區

安裝必要的 SDK 套件之後，現在您已準備好連線到您的工作區。

若要連線到工作區，我們需要識別碼參數 - 訂用帳戶識別碼、資源組名和工作區名稱。 資源組名和工作區名稱已為您填入。 您只需要訂用帳戶識別碼才能完成命令。

若要尋找必要的參數，請按一下 Studio 右上方的訂用帳戶和工作區名稱。 窗格會在右側開啟。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 複製訂用帳戶識別碼，並將 **YOUR-SUBSCRIPTION-ID** 取代為您複製的值。 </p>

## 以作業身分執行腳本

若要定型模型，您必須先在**src**資料夾中建立**diabetes_training.py**腳本。 腳本會使用與定型資料相同資料夾中 ** 的diabetes.csv** 檔案。

請注意，您會在腳本開頭匯入程式庫。 來自這些程式庫的函式可用來處理資料並定型模型。 您用來執行腳本的任何計算都必須安裝這些程式庫。

In [ ]:
建立腳本之後，您可以執行腳本作為作業。 腳本會使用通用程式庫。 因此，您可以使用包含 pandas、numpy 和 scikit-learn 等的策劃環境。

作業會使用最新版的策劃環境： `AzureML-sklearn-0.24-ubuntu18.04-py37-cpu` 。

In [ ]:
當作業正在執行時，您已經可以執行下一個資料格。

## 列出環境

讓我們探索工作區內的環境。 

在上一個工作中，您已使用其中一個策劃的環境。 若要探索工作區中已存在的所有環境，您可以列出環境： 

In [ ]:
請注意，所有策劃的環境都有開頭 **為 AzureML-** (您無法將此前置詞用於您自己的環境) 。

若要檢閱特定環境，您可以依其名稱和版本擷取環境。 例如，您可以擷取您用於上一個作業之策劃環境 *的描述* 和 *標記* ：

## 建立和使用自訂環境

如果策劃的環境不包含執行腳本所需的所有 Python 套件，您可以建立自己的自訂環境。 藉由列出環境中所有必要的套件，您可以輕鬆地重新執行腳本。 所有相依性都會儲存在環境中，然後您可以在作業組態中指定，而與您使用的計算無關。

例如，您可以直接從 Docker 映射建立環境。 PyTorch 之類的特定架構會有公用 Docker 映射，其中包含您需要的所有專案。 

讓我們從 Docker 映射建立環境：

In [ ]:
環境現在已在工作區中註冊，您可以在執行腳本作為作業時加以參考：

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 作業很快就會失敗！ 檢閱錯誤訊息。 </p>

錯誤訊息會告訴您沒有名為 pandas 的模組。 這類錯誤有兩個可能的原因：

- 腳本會使用 pandas，但未匯入程式庫 (`import pandas as pd`) 。 
- 腳本會在腳本頂端匯入程式庫，但計算未安裝程式庫 (`pip install pandas`) 。

檢 `diabetes-training.py` 閱腳本之後，您可以觀察腳本是否正確，這表示未安裝程式庫。 換句話說，環境不包含必要的套件。

讓我們使用上一個作業中使用的基底 Docker 映射來建立新的環境。 現在，您將新增 conda 規格，以確保將安裝必要的套件。 首先，執行下列儲存格來建立 conda 規格檔案：

請注意，conda 規格檔案中包含所有必要的相依性，腳本才能順利執行。

使用基底 Docker 映射 **和** conda 規格檔案建立新的環境，以新增必要的相依性。 Azure Machine Learning 會在您提供的 Docker 映射之上建置 conda 環境。 

In [ ]:
現在，您可以使用新的環境提交作業，以執行腳本：

使用新的自訂環境提交作業會觸發環境的組建。 第一次使用新建立的環境時，建置環境可能需要 10-15 分鐘的時間，這也表示您的作業需要較長的時間才能完成。 

您也可以選擇在提交作業之前手動觸發環境的組建。 您第一次使用環境時，只需要建置環境。 

當作業觸發新環境的組建時，您可以在作業的 [ **輸出 + 記錄** ] 索引標籤中檢閱組建的記錄。 開啟 **azureml-logs/20_image_build_log.txt** ，以檢查環境組建的記錄。 

![建置記錄的螢幕擷取畫面](./images/screenshot-logs.png)

In [ ]:
env = ml_client.environments.get("AzureML-sklearn-0.24-ubuntu18.04-py37-cpu", version=44)
print(env. description, env.tags)

## Create and use a custom environment

If a curated environment doesn't include all the Python packages you need to run your script, you can create your own custom environment. By listing all necessary packages in an environment, you can easily re-run your scripts. All the dependencies are stored in the environment which you can then specify in the job configuration, independent of the compute you use.

For example, you can create an environment simply from a Docker image. Certain frameworks like PyTorch will have a public Docker image that already includes everything you need. 

Let's create an environment from a Docker image:

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_image = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    name="docker-image-example",
    description="Environment created from a Docker image.",
)
ml_client.environments.create_or_update(env_docker_image)

The environment is now registered in your workspace and you can reference it when you run a script as a job:

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> The job will quickly fail! Review the error message. </p>

The error message will tell you that there is no module named pandas. There are two possible causes for such an error:

- The script uses pandas but didn't import the library (`import pandas as pd`). 
- The script does import the library at the top of the script but the compute didn't have the library installed (`pip install pandas`).

After reviewing the `diabetes-training.py` script you can observe the script is correct, which means the library wasn't installed. In other words, the environment didn't include the necessary packages.

Let's create a new environment, using the base Docker image used in the previous job. Now, you'll add a conda specification to ensure the necessary packages will be installed. First, run the following cell to create the conda specification file:

In [ ]:
%%writefile src/conda-env.yml
name: basic-env-cpu
channels:
  - conda-forge
dependencies:
  - python=3.7
  - scikit-learn
  - pandas
  - numpy
  - matplotlib

Note that all necessary dependencies are included in the conda specification file for the script to run successfully.

Create a new environment using the base Docker image **and** the conda specification file to add the necessary dependencies. Azure Machine Learning will build the conda environment on top of the Docker image you provided. 

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="./src/conda-env.yml",
    name="docker-image-plus-conda-example",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

Now, you can submit a job with the new environment to run the script:

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-plus-conda-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Submitting the job with the new custom environment triggers the build of the environment. The first time you use a newly created environment, it can take 10-15 minutes to build the environment, which also means your job will take longer to complete. 

You can also choose to manually trigger the build of the environment before you submit a job. The environment only needs to be built the first time you use it. 

When the job triggers the build of a new environment, you can review the logs of the build in the **Outputs + logs** tab of the job. Open **azureml-logs/20_image_build_log.txt** to inspect the logs of the environment build. 

![Screenshot build logs](./images/screenshot-logs.png)